In [2]:
# Importing packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Data Source
df1 = pd.read_csv('https://umbc.box.com/shared/static/au8h309qptksq5wlem6g5onn5v13fsrc.csv', encoding='ISO-8859-1')
df2=pd.read_csv('https://umbc.box.com/shared/static/tdkmuj86qm2uxhvjlfkav2i3pnx9fkoa.csv',encoding='ISO-8859-1')
df3=pd.read_csv('https://umbc.box.com/shared/static/7emabgf9ck2afprmrl459jnk30ewmabj.csv',encoding='ISO-8859-1')
df4=pd.read_csv('https://umbc.box.com/shared/static/91lmieun5u1e8p87wmlnuglyzl3xm91h.csv',encoding='ISO-8859-1')

# merged all the four data frames
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
#Cleaning the data

def clean_contract_data(df):
    # Clean 'Award$' column to extract numeric values
    df['Award$'] = pd.to_numeric(df['Award$'].str.replace('[^\d.]', ''), errors='coerce')

    # Convert the 'PostedDate' column into UTC format
    df['PostedDate'] = pd.to_datetime(df['PostedDate'], errors='coerce', utc=True)

    # Convert the 'Awardee' column to string type
    df['Awardee'] = df['Awardee'].astype(str)

    return df

# function with the contract data
combined_df = clean_contract_data(combined_df)

In [ ]:
combined_df.columns

# **1.Census Bureau has the largest number of contracts in Department of Commerce**

In [ ]:
#Filtering the combined_df on basis of Department/Ind.Agency as "Commerce"
com_df = combined_df[combined_df['Department/Ind.Agency'].str.contains('COMMERCE',na=False)]
com_df

In [ ]:
#Fetching the Census Bureau subtier data from Department/Ind.Agency as "Commerce"
census = com_df[com_df['Sub-Tier'].str.contains('US CENSUS BUREAU',na=False)]
census

In [ ]:
#Fectching the value count and resetting the index
compare = com_df['Sub-Tier'].value_counts().reset_index()
compare.rename(columns={'index':'Sub-Tier','Sub-Tier':'count'},inplace=True)
compare

In [ ]:
import plotly.express as px

# Creating bar graph
fig = px.bar(compare, y='Sub-Tier', x='count', orientation='h', color='Sub-Tier', text='count',
             title='Counts of Sub-Tier Entities',
             category_orders={"Sub-Tier": compare.sort_values("count", ascending=True)['Sub-Tier']})
fig.update_layout(width=960, height=640, showlegend=False, title_x=0.5, title_font_color='darkblue',
                  xaxis_tickfont_size=12, yaxis_tickfont_size=10,
                  margin=dict(l=40, r=40, t=60, b=40),  # Adjusted margins
                  plot_bgcolor='white', paper_bgcolor='white',
                  shapes=[
                      dict(type="line", x0=0, y0=-0.5, x1=0, y1=len(compare['Sub-Tier'])-0.5, line=dict(color="Black", width=2)),
                      dict(type="line", x0=min(compare['count']), y0=-0.5, x1=max(compare['count']), y1=-0.5, line=dict(color="Black", width=2))
                  ])

# Adjusting bar text
fig.update_traces(texttemplate='%{text}', textposition='outside').show()

Since 'NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION' has the highest number of contracts our hypothesis is wrong.

# **2. Department of Defense posted the highest number of contracts over the last several years comparing with other departments.**

In [ ]:
#Fectching the value count of Department/Ind.Agency
display = combined_df['Department/Ind.Agency'].value_counts()

#Restting the index and fecthing top 10
xyz = display.head(10).reset_index()
xyz.rename(columns={'index':'Department/Ind.Agency','Department/Ind.Agency':'Count'},inplace=True)
xyz

In [ ]:
import plotly.graph_objects as go

# Create a 3D pie chart
fig = go.Figure(data=[go.Pie(labels=xyz['Department/Ind.Agency'], values=xyz['Count'], textinfo='percent', pull=[0.1 if agency == "DEPT OF DEFENSE" else 0 for agency in xyz['Department/Ind.Agency']], hole=0.3)])

# Customize layout
fig.update_layout(title='Department/Agency Counts', showlegend=True, legend_title_text='Departments/Agencies',
                  title_font_color='darkblue', title_x=0.5,
                  font=dict(family="Arial, sans-serif", size=12, color='black'),
                  paper_bgcolor='white')

# Show the figure
fig.show()

Overall, our hypothesis that defense department has the highest number of contracts as compared to the other departments.


# **3. Over the last several years, Census Bureau posted the same number of contract over the years**

In [ ]:
import pandas as pd
import plotly.graph_objects as go
# Creating a Year column from the PostedDate column
combined_df['Year'] = combined_df['PostedDate'].dt.year

# Filter the Sub_tier "US CENSUS BUREAU" from combined_df
census = combined_df[combined_df['Sub-Tier'].str.contains('US CENSUS BUREAU', na=False)]

# Group by year and count the number of contracts
contracts_by_year = census.groupby('Year').size()

# Creating a DataFrame for Plotly
plotly_df = pd.DataFrame({'Year': contracts_by_year.index, 'Number of Contracts': contracts_by_year.values})

# Finding the year corresponding to the maximum value
max_value_year = plotly_df.loc[plotly_df['Number of Contracts'].idxmax(), 'Year']

# Plotting the trend using Plotly Express
fig = go.Figure()

# Adding a line trace
fig.add_trace(go.Scatter(
    x=plotly_df['Year'], y=plotly_df['Number of Contracts'],
    mode='lines+markers', line_shape='linear', marker=dict(symbol='circle', size=8),
    name='Number of Contracts', line=dict(color='blue', width=2)
))

# Highlighting maximum value with a larger marker and bold label
max_value = contracts_by_year.max()
fig.add_trace(go.Scatter(
    x=[max_value_year], y=[max_value], mode='markers+text',
    marker=dict(color='red', size=12), text=[f'Max: {max_value}'],
    textposition='bottom right', textfont=dict(color='red', size=12)
))

# Adding a background color
fig.add_shape(
    type='rect', x0=min(plotly_df['Year']), x1=max(plotly_df['Year']), y0=0, y1=max_value,
    fillcolor='lightgrey', opacity=0.2, layer='below', line_width=0
)

# Adding hover text
fig.update_traces(
    hovertemplate='%{y} contracts<br>%{x}'
)

# Adding title and labels with grid customization
fig.update_layout(
    title='Number of Contracts Posted by Census Bureau Over the Years',
    xaxis=dict(title='Year', showgrid=True, gridcolor='grey', gridwidth=2, tickmode='linear', dtick=1),
    yaxis=dict(title='Number of Contracts', showgrid=True, gridcolor='grey', gridwidth=2),
    legend=dict(font=dict(size=12)),
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='black'),
    hovermode='x'
)

# Highlighting titles
fig.update_layout(
    xaxis_title_font=dict(size=14, family='Arial', color='darkblue'),
    yaxis_title_font=dict(size=14, family='Arial', color='darkblue')
)

# Show the plot
fig.show()

From above we can see an increase in number of contracts evrey year. The change is less but not negligible. Hence our hypothesis is wrong.

# **4. There is an increase in number of contract posted for Department of Commerce over the last several years**

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Creating a Year column from the PostedDate column
combined_df['Year'] = combined_df['PostedDate'].dt.year

# Filtering contracts for the Department of Commerce
commerce_contracts = combined_df[combined_df['Department/Ind.Agency'] == 'COMMERCE, DEPARTMENT OF']

# Grouping by year and counting the number of contracts
contracts_by_year = commerce_contracts.groupby('Year').size()

# Finding the year corresponding to the maximum value
max_value_year = contracts_by_year.idxmax()

# Creating a DataFrame for Plotly
plotly_df = pd.DataFrame({'Year': contracts_by_year.index, 'Number of Contracts': contracts_by_year.values})

# Plotting the trend using Plotly
fig = px.line(plotly_df, x='Year', y='Number of Contracts', markers=True, line_shape='linear', title='Number of Contracts Posted by Department of Commerce Over the Years',
              labels={'Number of Contracts': 'Number of Contracts'},
              template='plotly_dark')

# Force x-axis ticks to display only integer years
fig.update_layout(xaxis=dict(tickmode='array', tickvals=contracts_by_year.index.astype(int), ticktext=contracts_by_year.index.astype(int).astype(str)))

# Adding annotations with a more visible color
for i, value in enumerate(contracts_by_year.values):
    fig.add_annotation(x=contracts_by_year.index[i], y=value, text=str(value), showarrow=True, arrowhead=4, ax=0, ay=-40, font=dict(color='white', size=12))

# Highlighting maximum value with a larger marker and bold label
max_value = contracts_by_year.max()
fig.add_trace(go.Scatter(x=[max_value_year], y=[max_value], mode='markers+text', marker=dict(color='red', size=12), text=[f'Max: {max_value}'],
                         textposition='bottom right', textfont=dict(color='red', size=12)))

# Adding a background color for a more interactive feel
fig.add_shape(type='rect', x0=min(contracts_by_year.index), x1=max(contracts_by_year.index), y0=0, y1=max_value, fillcolor='lightgrey', opacity=0.2, layer='below', line_width=0)

# Customizing layout
fig.update_layout(xaxis=dict(title='Year', showgrid=True, gridcolor='grey'),
                  yaxis=dict(title='Number of Contracts', showgrid=True, gridcolor='grey'),
                  legend=dict(font=dict(size=12)),
                  showlegend=False)  # Set showlegend to True if you want to display legend

fig.show()


WE can see a sharp increase after 2019 and minor growth from 2020. Indicating number of contracts increased every year. Hence, the hypotheses stands true.